# GDA Implementation.

Implement the Gaussian Discriminant Analysis (GDA) learning algorithm following the steps as discussed in class.

INSTRUCTION: Rename your notebook as: <br>
`firstName_LastName_Live_coding_GDA.ipynb`.

Notes: 
* Do not use any built-in functions to complete a task;
* Do not import additional libraries.

In [18]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

In [19]:
# Generate data
def generate_data():
  x, y = make_classification(n_samples= 1000, n_features=3, n_redundant=0, 
                           n_informative=3, random_state=1, 
                           n_clusters_per_class=1)
  
  return x,y

x,y= generate_data() # get data
print(x.shape, y.shape)

(1000, 3) (1000,)


In [20]:
def split_data(x,y, train_size= 0.8):
    # shuffle the data to randomize the train/test split
    indices = np.random.permutation(len(y))
    
    # Split the shuffled indices into training and testing sets
    train_indices = indices[:int(train_size*len(y))]
    test_indices = indices[int(train_size*len(y)):]
    
    # Use the shuffled indices to extract the corresponding data
    x_train, x_test = x[train_indices], x[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]
    return x_train, x_test, y_train, y_test

In [21]:
X_train, X_test, y_train, y_test= split_data(x, y, train_size=0.8)# split your data into x_train, x_test, y_train, y_test
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(800, 3) (800,) (200, 3) (200,)


In [22]:
def covariance(x, mu):
    d = x.shape[1]
    mu=np.mean(x,axis=0)
    cov = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            cov[i, j] = np.sum((x[:,i] - mu[i]) * (x[:,j] - mu[j])) / (len(x) - 1)
    return cov

In [23]:
print(np.mean(x,axis=0))

[-0.01298493  0.99925415  0.02975341]


In [24]:
mu=np.mean(x,axis=0)
covariance(x, mu)

array([[1.84495325, 0.02790646, 1.00137533],
       [0.02790646, 1.00170721, 0.05539176],
       [1.00137533, 0.05539176, 1.74832   ]])

In [25]:
np.cov(x,rowvar=0)

array([[1.84495325, 0.02790646, 1.00137533],
       [0.02790646, 1.00170721, 0.05539176],
       [1.00137533, 0.05539176, 1.74832   ]])

In [26]:
class GDA:
  def __init__(self):
    ## set mu, phi and sigma to None
    self.mu=None
    self.phi=None
    self.sigma=None
    
  def fit(self,x,y):
    k=2 #np.unique(y).size  # Number of class.
    d=x.shape[1] # input dim
    m= x.shape[0]# Number of examples.
    
    ## Initialize mu, phi and sigma
    self.mu= np.zeros((k,d))#: kxd, i.e., each row contains an individual class mu.
    self.sigma= np.zeros((k,d,d))#: kxdxd, i.e., each row contains an individual class sigma.
    self.phi= np.zeros((k))# d-dimension

    ## START THE LEARNING: estimate mu, phi and sigma.
    for lab in range(k):
        
        self.phi[lab] = np.sum(lab==y)/m 
        self.mu[lab] = np.mean(x[lab==y], axis=0)
        self.sigma[lab] = covariance(x[lab==y], self.mu[lab])
    return self.phi,self.mu, self.sigma
            
            

  def predict_proba(self,x):
    # reshape or flatt x.
    #x= x.reshape(-1, self.mu.shape[1])
    #x=x.reshape(-1,1)
    #x=self.mu.shape[0]
    d= x.shape[0]
    #k_class = self.mu.shape[0] 
    k_class= self.mu.shape[0]  # Number of classes we have in our case it's k = 2
    probabilities = np.zeros((d, k_class))
    
    ## START THE LEARNING: estimate mu, phi and sigma.
    for lab in range(k_class):
        det_cov= np.linalg.det(self.sigma[lab])
        inv_cov=np.linalg.inv(self.sigma[lab])
        for j in range(x.shape[0]):
            first_term=1/((2*np.pi)**(d/2)*(det_cov**0.5))
            #first_term=((1/(((2*np.pi)**(d/2))*(det_cov**0.5)))
            exponential=-0.5*((x[j]-self.mu[lab]).T)@(inv_cov)@(x[j]-self.mu[lab])
            second_term=np.exp(exponential)
            probabilities[j, lab] = first_term*second_term*self.phi[lab]
    return probabilities

  def predict(self,x):
    predict=self.predict_proba(x)
    
    y = np.argmax(predict,axis = 1)
    #Predict = np.argmax(self.predict_proba(x))
    return y
    
  
  def accuracy(self, y, ypreds):
    ypred = self.predict(y)
    result = np.mean(ypred == ypreds)
    return result * 100 

In [27]:
model= GDA()
model.fit(X_train,y_train)

(array([0.48625, 0.51375]),
 array([[ 0.95351593,  1.08279465,  1.00647428],
        [-1.00852009,  0.95000511, -0.96940467]]),
 array([[[ 0.8833787 , -0.35895027, -0.05927973],
         [-0.35895027,  1.74795885,  0.10243629],
         [-0.05927973,  0.10243629,  0.03800151]],
 
        [[ 0.82756154,  0.33795434,  0.14443228],
         [ 0.33795434,  0.3544237 , -0.06028125],
         [ 0.14443228, -0.06028125,  1.63577278]]]))

In [28]:
yproba= model.predict_proba(X_test)
yproba

array([[7.38627506e-081, 1.02846675e-083],
       [3.12922214e-082, 2.31363438e-082],
       [1.65690429e-080, 4.66650070e-086],
       [1.78865136e-081, 1.43146791e-090],
       [1.52637158e-113, 3.12990897e-081],
       [1.55241893e-110, 1.47304241e-080],
       [1.14969763e-080, 2.78683256e-083],
       [3.01298951e-081, 3.15431759e-084],
       [1.67275050e-205, 3.53856746e-081],
       [1.37339640e-080, 1.38405131e-089],
       [9.24173071e-107, 1.00128015e-080],
       [1.67389531e-081, 4.80992590e-089],
       [1.70705027e-080, 2.66269494e-082],
       [1.47916469e-147, 3.84495004e-081],
       [3.02644674e-081, 1.78582923e-082],
       [2.35373762e-080, 1.39970217e-082],
       [3.73980723e-137, 8.45121556e-081],
       [1.52390564e-092, 6.60723389e-081],
       [5.68728343e-086, 7.90612459e-081],
       [4.84928231e-081, 2.51137114e-087],
       [2.23849876e-095, 8.73481650e-081],
       [2.12139496e-080, 1.42197489e-082],
       [1.40276516e-094, 1.42878846e-081],
       [7.6

In [29]:
ypreds= model.predict(X_test)
ypreds


array([0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0,
       0, 1])

In [30]:
model.accuracy(y_test, ypreds)

2.5